<a href="https://colab.research.google.com/github/ThatiFlowers/Treinamento/blob/main/C%C3%B3pia_de_CRUD_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando sql e criando conexões e cursores

import sqlite3

def conexao_bd():
    conexao = sqlite3.connect("cafe_porquinho.db")
    return conexao

def create_tables():
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        preco REAL NOT NULL,
        categoria TEXT NOT NULL
    )
    """)

    conexao.commit()
    conexao.close()

create_tables()


In [ ]:
from os import name
# Criando a classe de produtos/ repositório(domínio)
class Produto:
    def __init__(self,nome,preco,categoria):
        self.nome = nome
        self.preco = preco
        self.categoria = categoria
    def __repr__(self):
        return f"Produto(id={self.id}, nome='{self.nome}', preco={self.preco}, categoria='{self.categoria}')"

In [ ]:
# Criando uma pasta no ambiente notebook
!mkdir -p repository

In [ ]:
def cadastrar_ produto(produto):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute(
        "INSERT INTO produtos (nome, preco, categoria) VALUES (?, ?, ?)",
        (produto.nome, produto.preco, produto.categoria)
    )

    conexao.commit()
    conexao.close()


In [ ]:
def limpar_produtos():
    conexao = conexao_bd()
    cursor = conexao.cursor()
    cursor.execute("DELETE FROM produtos")
    conexao.commit()
    conexao.close()



In [ ]:
#teste

produto1 = Produto("cocavinho",12, "bebida")
produto2 = Produto("bolorosa",12.59,"comida")
produto3 = Produto("brownie", 11, "comida")
cadastrar_produto(produto1)
cadastrar_produto(produto2)
cadastrar_produto(produto3)

In [ ]:
def listar_produtos():
  conexao = conexao_bd()
  cursor = conexao.cursor()
  cursor.execute("select * from produtos")
  rows = cursor.fetchall()
  conexao.close()
  return rows

print("Produtos cadastrados:")
for produto_data in listar_produtos():
    print(f"ID: {produto_data[0]}, Nome: {produto_data[1]}, Preço: {produto_data[2]}, Categoria: {produto_data[3]}")

Produtos cadastrados:
ID: 1, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 2, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 3, Nome: brownie, Preço: 11.0, Categoria: comida
ID: 4, Nome: cocavinho, Preço: 12.0, Categoria: bebida
ID: 5, Nome: bolorosa, Preço: 12.59, Categoria: comida
ID: 6, Nome: brownie, Preço: 11.0, Categoria: comida


In [ ]:
for p in listar_produtos():
    print(p[1], p[2], p[3])

cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida
cocavinho 12.0 bebida
bolorosa 12.59 comida
brownie 11.0 comida


In [ ]:
# covertendo em uma lista de produtos
def converter_rows_para_produtos(rows):
    lista_de_produtos = []
    for row in rows:
        # formato: (id, nome, preco, categoria)
        p = Produto(row[1], row[2], row[3])
        lista_de_produtos.append(p)
    return lista_de_produtos

def produtos_id(id):
    conexao = conexao_bd()
    cursor = conexao.cursor()

    cursor.execute("select * from produtos where id =?", (id,))
    row = cursor.fetchone() # Changed fetchall to fetchone as we expect a single product by ID
    conexao.close()
    if row:
        p = Produto(row[1], row[2], row[3])
        return p
    return None # Retorna "none" se o produto for encontrado com ID conhecido

In [ ]:
# Crie uma pasta 'models'no ambiente notebook
!mkdir -p models

In [ ]:
# Criando o Flask para obter o API

In [ ]:
pip install Flask

In [8]:
from flask import Flask, request, jsonify

app = Flask(__name__)

produtos = [
    {
        'id': 1,
        'bebida': 'cocavinho',
        'comida': 'bolorosa'
    },
    {
       'id': 2,
        'bebida': 'cocabaunilha',
        'comida': 'bolo arco-irís'
    }
]

#consultar todos
@app.route('/produtos', methods=['GET'])
def obter_produtos():
    return jsonify(produtos)


#consultar por id
@app.route('/produtos/<int:id>', methods=['GET'])
def obter_produtos_por_id(id):
  for produto in produtos:
    if produto.get('id') == id:
      return jsonify(produto)
  return jsonify({'message': 'Produto não encontrado'}), 404

#editar
@app.route('/produtos/<int:id>', methods=['PUT'])
def editar_produto_por_id(id):
  produto_alterado = request.get_json()
  for indice, produto in enumerate(produtos):
    if produto.get('id') == id:
      produtos[indice].update(produto_alterado)
      return jsonify(produtos[indice])
  return jsonify({'message': 'Produto não encontrado para atualização'}), 404

 #criar
@app.route('/produtos', methods=['POST'])
def criar_produto():
  novo_produto = request.get_json()
  produtos.append(novo_produto)
  return jsonify(produtos)


#excluir
@app.route('/produtos/<int:id>', methods=['DELETE'])
def excluir_produto(id):
  for indice, produto in enumerate(produtos):
    if produto.get('id') == id:
      del produtos[indice]
      return jsonify({'message': 'Produto excluído com sucesso', 'produtos_restantes': produtos})
  return jsonify({'message': 'Produto não encontrado para exclusão'}), 404


In [10]:
from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)

@app.route('/produtos', methods=['GET'])
def listar():
    return jsonify([{"id": 1, "nome": "Café", "preco": 5.0}])

def run():
    app.run(port=5001)

thread = Thread(target=run)
thread.start()
